<a href="https://colab.research.google.com/github/Edimer/My_CloudComputing/blob/master/AirQoPCAPower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AirQo Ugandan Air Quality Forecast Challenge
===========================================
Sidereus

## Packages

In [0]:
#install.packages("h2o", dependencies = TRUE)
library(tidyverse)
library(h2o)

## Import data

In [0]:
load("/content/dataTest_Originales.Rdata")
load("/content/dataTrain_Originales.Rdata")

## PCA (Method = "power")

- [Documentation Principal Components with h2o.](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/pca.html)

### Starting h2o and data in format h2o

In [4]:
# Starting h2o
h2o.init(nthreads = -1, port = 54321, max_mem_size = "10g")

# Train data
dataTrainPCA <- dataTrain %>% select(-c(ID, target))
targetTrain <- dataTrain %>% pull(target)

# Test data
dataTestPCA <- dataTest %>% select(-ID)

# Train and test in format h2o
trainH2o <- as.h2o(dataTrainPCA)
testH2o <- as.h2o(dataTestPCA)

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         35 minutes 8 seconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.30.0.1 
    H2O cluster version age:    1 month and 7 days  
    H2O cluster name:           H2O_started_from_R_root_taq814 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   9.60 GB 
    H2O cluster total cores:    2 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, TargetEncoder, Core V4 
    R Version:                  R version 3.6.3 (2020-02-29) 

  |======================================================================| 100%
  |=============================================================

### Model PCA with method="power"

- In this model I use imputation.

In [5]:
modelPCA <- h2o.prcomp(training_frame = trainH2o, transform = "STANDARDIZE",
                       k = 9, pca_method = "Power", use_all_factor_levels = TRUE,
                       impute_missing = TRUE, model_id = "PCA_Power", seed = 123,
                       max_runtime_secs = 3600)

Warning message in .h2o.processResponseWarnings(res):
“Dropping bad and constant columns: [precMinTotal].
”


  |======================================================================| 100%


- **Model summary:**

In [6]:
modelPCA@model$importance

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Standard deviation,12.0908226,9.26767192,8.95476583,8.77588307,4.73614263,3.74440670,3.5924675,3.4259820,2.997724991
Proportion of Variance,0.1624311,0.09543304,0.08909758,0.08557346,0.02492338,0.01557842,0.0143398,0.0130415,0.009984838
Cumulative Proportion,0.1624311,0.25786413,0.34696171,0.43253518,0.45745856,0.47303698,0.4873768,0.5004183,0.510403126


- **New data with PCA:**

In [8]:
# Train
trainPCA <- h2o.predict(modelPCA, trainH2o)
testPCA <- h2o.predict(modelPCA, testH2o)

  |======================================================================| 100%
  |======================================================================| 100%


- **Data to R:**

In [0]:
# Train
dfTrain <- as.data.frame(trainPCA)
dfTrain$target <- targetTrain

# Test
dfTest <- as.data.frame(testPCA)
dfTest$ID <- dataTest$ID

- **Export data:**

In [0]:
save(dfTrain, file = "trainPCAPower.Rdata", compress = "xz")
save(dfTest, file = "testPCAPower.Rdata", compress = "xz")